In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
X = np.array([ 
    [180, 12.5],
    [200, 13.0],
    [220, 13.5],
    [250, 14.0],
    [160, 12.0],
    [240, 12.2],
    [210, 13.2],
    [190, 13.8],
    [170, 15.0],
    [260, 11.5]
    ],dtype = float)

# 1 is good roast and 0 is bad roast
y = np.array([1, 1, 1, 0, 0, 1, 1, 1, 0, 0],dtype = float) 

In [3]:
X.shape

(10, 2)

In [4]:
y.shape

(10,)

In [5]:
y

array([1., 1., 1., 0., 0., 1., 1., 1., 0., 0.])

In [6]:
# normalization
norm_1 = tf.keras.layers.Normalization(axis=-1)
norm_1.adapt(X)
Xn = norm_1(X)
print(f"tem max and min post normalization: {np.max(Xn[:,0])} , {np.max(Xn[:,0])}")
print(f"Duration max and min post normalization: {np.max(Xn[:,1])} , {np.max(Xn[:,1])}")

tem max and min post normalization: 1.6001893281936646 , 1.6001893281936646
Duration max and min post normalization: 1.9279768466949463 , 1.9279768466949463


In [7]:
# Tile/copy our data to increase the training set size and reduce the number of training epochs.
Xt = np.tile(Xn,(100,1))
yt = np.tile(y,(100,))
print(Xt.shape,yt.shape)

(1000, 2) (1000,)


TensorFlow model

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


tf.random.set_seed(1234)
model = Sequential(
    [
        tf.keras.Input(shape=(2,)),
        Dense(3, activation='sigmoid', name = 'layer1'),
        Dense(1, activation='sigmoid', name = 'layer2')
     ]
)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 layer2 (Dense)              (None, 1)                 4         
                                                                 
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [10]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print(f"W1{W1.shape}:\n", W1, f"\nb1{b1.shape}:", b1)
print(f"W2{W2.shape}:\n", W2, f"\nb2{b2.shape}:", b2)

W1(2, 3):
 [[ 1.0194116  -0.68084085  0.807464  ]
 [-0.5144466  -0.3685394   0.9676111 ]] 
b1(3,): [0. 0. 0.]
W2(3, 1):
 [[-0.3953902 ]
 [-0.5750476 ]
 [-0.11119318]] 
b2(1,): [0.]


In [12]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

)
model.fit(
    Xt,yt,epochs = 10,
)

Epoch 1/10
32/32 [==============================] - 1s 2ms/step - loss: 0.7261
Epoch 2/10
32/32 [==============================] - 0s 2ms/step - loss: 0.6780
Epoch 3/10
32/32 [==============================] - 0s 2ms/step - loss: 0.6733
Epoch 4/10
32/32 [==============================] - 0s 2ms/step - loss: 0.6706
Epoch 5/10
32/32 [==============================] - 0s 2ms/step - loss: 0.6671
Epoch 6/10
32/32 [==============================] - 0s 2ms/step - loss: 0.6596
Epoch 7/10
32/32 [==============================] - 0s 2ms/step - loss: 0.6485
Epoch 8/10
32/32 [==============================] - 0s 2ms/step - loss: 0.6304
Epoch 9/10
32/32 [==============================] - 0s 4ms/step - loss: 0.6073
Epoch 10/10
32/32 [==============================] - 0s 2ms/step - loss: 0.5775


In [13]:
W1,b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

W1:
 [[ 3.042228  -1.5388355  0.5555355]
 [ 0.6343955  0.6784215  2.8910275]] 
b1: [-2.4650595 -1.0055321  2.4081407]
W2:
 [[-1.1983109]
 [-0.5873111]
 [ 0.7646746]] 
b2: [0.39627308]


In [17]:
X_test = np.array([
    [200,13.9],
    [100,11]
])
X_testn = norm_1(X_test)
prediction = model.predict(X_testn)
print('predictions = \n',prediction)


1/1 [==============================] - 0s 31ms/step
predictions = 
 [[0.6885372 ]
 [0.46235272]]


In [18]:
yhat = (prediction >= 0.5 ).astype(int)
print(f'decisions = \n',yhat)

decisions = 
 [[1]
 [0]]
